In [14]:
import requests as res
from tqdm.notebook import tqdm

In [10]:
TOKEN = ''
ENDPOINT = 'https://solomon.yandex-team.ru/api/v2/projects/yandexcloud/sensors/data'

LABELS_LIMIT = 100000
HEADERS = {
    'Content-Type': 'application/json',
    'User-Agent': 'YC DWH',
    'Accept': 'application/json',
    'Authorization': f'OAuth {TOKEN}'
}
LABEL_URL = 'https://solomon.yandex-team.ru/api/v2/projects/yandexcloud/sensors/labels'


In [11]:
params_for_hosts = {'names': ['host'], 'limit': 100000, 'selectors': 'cluster="cloud_prod_compute",service="compute",metric="cpu-util"'}
hosts_req = res.get(url=LABEL_URL, params=params_for_hosts, headers=HEADERS)
hosts = hosts_req.json()['labels'][0]['values']

In [13]:
hosts_chunks = [hosts[x:x + 100] for x in range(0, len(hosts), 100)]

In [25]:
instance_id_list = []
for host in tqdm(hosts_chunks):
    params_for_instances = {
        'names': ['instance_id'],
        'limit': 100000,
        'selectors': f'cluster="cloud_prod_compute",project="yandexcloud",service="compute",metric="cpu-util",host=~{"|".join(host)}'
    }
    r = res.get(url=LABEL_URL, params=params_for_instances, headers=HEADERS)
    print(len(r.json()['labels'][0]['values']))
    instance_id_list += r.json()['labels'][0]['values']

  0%|          | 0/68 [00:00<?, ?it/s]

8343
12415
13650
15357
4060
4676
3307
20102
8400
17327
13313
14566
17204
13885
14644
10775
14878
9112
6361
4907
6530
4548
4390
4595
5166
3447
7523
6155
5322
3993
1700
15271
15122
17864
16751
10467
7718
9483
20168
11334
4754
4955
4084
4811
3861
4755
5221
6682
6788
7513
6952
6283
4857
12719
5936
4620
28027
10134
1648
28708
9824
18025
16139
4549
14387
13251
29679
5997


In [16]:
import yt.wrapper as yt

In [17]:
YT_TOKEN = ''
yt.config["proxy"]["url"] = "hahn"
yt.config["token"] = YT_TOKEN

In [28]:
path = '//home/cloud-dwh/data/prod/raw/solomon/yandexcloud_cpu_utilization/cpu-util'
already_uploaded_path = '//home/cloud-dwh/data/prod/raw/solomon/yandexcloud_cpu_utilization/already_uploaded_instances'

In [29]:
already_uploaded = set()
for row in yt.read_table(already_uploaded_path, format=yt.JsonFormat()):
    already_uploaded.add(row['instance_id'])

2022-05-04 12:10:24,457	INFO	Table chunks are too small; consider running the following command to improve read performance: yt merge --proxy hahn --src //home/cloud-dwh/data/prod/raw/solomon/yandexcloud_cpu_utilization/already_uploaded_instances --dst //home/cloud-dwh/data/prod/raw/solomon/yandexcloud_cpu_utilization/already_uploaded_instances --spec '{combine_chunks=true;}'


In [30]:
err_instances = list(set(instance_id_list) - already_uploaded)

In [39]:
err_instances[0]

'epd2f8hv6cemes10nfh4'

In [69]:
host_list = {}
for instance in tqdm(err_instances):
    params_for_host_2 = {
        'names': ['host'],
        'limit': 100000,
        'selectors': f'cluster="cloud_prod_compute",project="yandexcloud",service="compute",metric="cpu-util",instance_id={instance}'
    }
    r = res.get(url=LABEL_URL, params=params_for_host_2, headers=HEADERS)
    print(len(r.json()['labels'][0]['values']))
    host_list[instance] = r.json()['labels'][0]['values']

  0%|          | 0/757 [00:00<?, ?it/s]

1
1
29
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
7
1
1
1
1
2
12
1
28
1
25
1
1
1
38
1
1
1
1
1
1
1
1
1
1
1
7
10
1
1
1
25
1
1
1
39
1
1
1
1
30
1
1
1
1
9
1
1
1
1
1
1
1
1
1
1
1
1
41
1
1
1
1
9
1
1
1
2
9
1
8
1
1
2
1
2
1
1
1
1
1
12
1
1
1
20
1
53
1
12
1
1
1
32
1
1
1
1
1
1
1
1
10
1
1
1
1
1
1
1
1
1
1
1
1
1
1
11
1
1
2
34
1
1
28
1
1
1
1
1
1
1
1
1
2
1
1
1
1
9
2
1
1
1
76
1
1
1
1
1
1
1
9
1
1
1
1
26
1
1
1
1
1
1
1
1
1
1
1
1
1
31
1
1
1
5
1
21
1
36
11
1
1
1
1
1
1
1
1
9
1
1
1
1
1
1
18
1
1
1
1
1
1
1
1
1
6755
1
1
1
1
33
1
1
1
1
1
2
27
1
1
1
1
24
33
1
1
1
2
1
1
1
1
1
1
1
2
1
1
18
1
1
1
1
1
1
1
18
7
1
1
1
1
1
1
1
1
1
1
7
1
1
11
1
8
1
1
1
1
18
1
1
1
1
1
1
8
1
1
1
1
1
9
1
1
1
1
21
1
1
1
1
1
39
1
1
159
1
1
1
1
1
1
1
1
1
1
1
9
1
1
1
1
1
19
1
1
43
1
1
1
1
11
1
1
1
27
1
1
1
10
1
1
1
2
1
20
1
1
1
1
1
1
1
1
27
1
1
1
1
9
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
7
1
1
1
1
11
1
1
1
1
1
1
1
1
10
129
1
1
28
1
1
1
1
11
1
1
1
1
1
1
1
1
1
44
1
1
1
155
3
36
1
1
1
11
1
1
1
1
1
1
1
1
1
1
11
1
1
1
1
1
1
1
1
1
1
1
1


In [70]:
host_list

{'epd2f8hv6cemes10nfh4': ['sas09-3ct24-5a'],
 'ef31lof6080l92jukran': ['myt3-ct0-26b'],
 'epdonlmiq3j3iav5hqua': ['sas09-2ct1-31',
  'sas09-2ct10-33',
  'sas09-2ct10-37',
  'sas09-2ct17-5b',
  'sas09-2ct17-26a',
  'sas09-2ct17-28a',
  'sas09-2ct27-15a',
  'sas09-3ct16-13b',
  'sas09-3ct17-7a',
  'sas09-3ct17-9a',
  'sas09-3ct17-15b',
  'sas09-3ct18-32b',
  'sas09-3ct20-13b',
  'sas09-3ct21-13b',
  'sas09-3ct23-15a',
  'sas09-3ct25-5a',
  'sas09-3ct25-15a',
  'sas09-3ct25-26a',
  'sas09-3ct27-11b',
  'sas09-3ct29-24a',
  'sas09-4ct0-9b',
  'sas09-ct35-6',
  'sas09-ct37-5',
  'sas09-ct37-8',
  'sas09-ct37-9',
  'sas09-ct37-11',
  'sas09-ct37-40',
  'sas09-ct39-7',
  'sas09-ct41-30'],
 'fhmb4jvfndio9g1drvic': ['vla04-4ct15-7'],
 'fhm3vgr615eki3ilbb05': ['vla04-ct9-8'],
 'fhme1dtetvmb4qsks0us': ['vla04-2ct6-18'],
 'epd06g3ckugnej2086rn': ['sas09-ct13-13', 'sas09-ct25-10'],
 'ef3ss62fv88et12vv0i5': ['myt1-ct15-41'],
 'fhm5duma1fd57rlqkhva': ['vla04-ct25-36'],
 'epd9cdh7a0lntae22mpk': ['sas0

In [41]:
import datetime
import pytz
MSK_TIMEZONE = 'Europe/Moscow'
MSK_TIMEZONE_OBJ = pytz.timezone(MSK_TIMEZONE)


In [76]:
final_result = []
for instance_id in tqdm(host_list):
    for host_id in host_list[instance_id]:
        params = {
            'from': int(datetime.datetime.strptime("2020-01-01T00:00:00",
                                                     '%Y-%m-%dT%H:%M:%S').astimezone(MSK_TIMEZONE_OBJ).timestamp() * 1000),
            'program': f'''{{project='yandexcloud',cluster='cloud_prod_compute',service='compute',metric='cpu-util',instance_id='{instance_id}', host='{host_id}'}}''',
            'to': int(datetime.datetime(2022, 1, 12, tzinfo=MSK_TIMEZONE_OBJ).timestamp() * 1000)
                  }
        r = res.post(url=ENDPOINT, headers=HEADERS, params=None, json=params)
        result = r.json()
        for sensor in result['vector']:
            if 'timeseries' not in sensor:
                continue

            sensor = sensor['timeseries']

            if not sensor.get('values'):
                continue

            if 'value' in sensor['labels'] or 'timestamp' in sensor['labels']:
                raise KeyError(f'There are system fields in solomon data {sensor["labels"]}')

            for timestamp, value in zip(sensor['timestamps'], sensor['values']):
                data = sensor['labels'].copy()
                data['_meta'] = {
                    'kind': sensor.get('kind'),
                    'type': sensor.get('type'),
                }

                data['timestamp'] = timestamp
                data['value'] = str(value)
                final_result.append(data)

  0%|          | 0/757 [00:00<?, ?it/s]

In [82]:
final_result[0]

{'cluster': 'cloud_prod_compute',
 'cpu_name': 'cpu_11',
 'instance_id': 'fhmlulfje7d1atldh5es',
 'metric': 'cpu-util',
 'service': 'compute',
 'host': 'vla04-s9-33',
 'project': 'yandexcloud',
 'cloud_id': 'b1gt6t86j987p9mq1elg',
 'folder_id': 'b1gmqpcuofc8i0hv5ug8',
 '_meta': {'kind': 'DGAUGE', 'type': 'DGAUGE'},
 'timestamp': 1577857200000,
 'value': 'NaN'}

In [83]:

yt.write_table(
    table=yt.TablePath(path, append=True),
    format=yt.JsonFormat(attributes={"encode_utf8": False}),
    input_stream=final_result,
)

In [84]:
instances = [{'instance_id': x} for x in err_instances]
yt.write_table(
    yt.TablePath(already_uploaded_path, append=True),
    instances,
    format="json"
)